In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Sentiment Analysis/emails.csv')
dataset.head(10)

,text,spam,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109
0,Subject: naturally irresistible your corporate...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Subject: the stock trading gunslinger fanny i...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Subject: unbelievable new homes made easy im ...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Subject: 4 color printing special request add...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Subject: do not have money , get software cds ...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Subject: great nnews hello , welcome to medzo...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Subject: here ' s a hot play in motion homela...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Subject: save your money buy getting this thin...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Subject: undeliverable : home based business f...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [4]:
sentences = dataset['text'].tolist()
labels= dataset['spam'].tolist()

In [5]:
print(len(labels))

5730


In [6]:
#Preparing Data sets
size = int(len(sentences)*0.8)

sentences_trainings = sentences[0:size]
sentences_test =sentences[size:]

labels_trainings =labels[0:size]
labels_test=labels[size:]


#Make labels into numpy arrays for use with the network later
training_labels_final = np.array(labels_trainings)
testing_labels_final = np.array(labels_test)
print(training_labels_final.shape)
print(testing_labels_final.shape)




(4584,)
(1146,)


In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

def review_to_words(words):
  nltk.download("stopwords", quiet=True)
  stemmer= PorterStemmer()
  stopword = set(stopwords.words('english'))

  words = re.sub(r"[^a-zA-Z0-9]", " ", words.lower())
  words = words.split()
  words = [w for w in words if w not in stopword]
  words = [PorterStemmer().stem(w) for w in words]
  
  return words



In [8]:
print(sentences[1])
print(review_to_words(sentences[1]))

Subject: the stock trading gunslinger  fanny is merrill but muzo not colza attainder and penultimate like esmark perspicuous ramble is segovia not group try slung kansas tanzania yes chameleon or continuant clothesman no  libretto is chesapeake but tight not waterway herald and hawthorn like chisel morristown superior is deoxyribonucleic not clockwork try hall incredible mcdougall yes hepburn or einsteinian earmark no  sapling is boar but duane not plain palfrey and inflexible like huzzah pepperoni bedtime is nameable not attire try edt chronography optima yes pirogue or diffusion albeit no 
['subject', 'stock', 'trade', 'gunsling', 'fanni', 'merril', 'muzo', 'colza', 'attaind', 'penultim', 'like', 'esmark', 'perspicu', 'rambl', 'segovia', 'group', 'tri', 'slung', 'kansa', 'tanzania', 'ye', 'chameleon', 'continu', 'clothesman', 'libretto', 'chesapeak', 'tight', 'waterway', 'herald', 'hawthorn', 'like', 'chisel', 'morristown', 'superior', 'deoxyribonucl', 'clockwork', 'tri', 'hall', 'in

In [9]:
  words_train = [review_to_words(sentences) for sentences in sentences_trainings]
  words_test = [review_to_words(sentences) for sentences in sentences_test]

In [10]:
words_train[1]

['subject',
 'stock',
 'trade',
 'gunsling',
 'fanni',
 'merril',
 'muzo',
 'colza',
 'attaind',
 'penultim',
 'like',
 'esmark',
 'perspicu',
 'rambl',
 'segovia',
 'group',
 'tri',
 'slung',
 'kansa',
 'tanzania',
 'ye',
 'chameleon',
 'continu',
 'clothesman',
 'libretto',
 'chesapeak',
 'tight',
 'waterway',
 'herald',
 'hawthorn',
 'like',
 'chisel',
 'morristown',
 'superior',
 'deoxyribonucl',
 'clockwork',
 'tri',
 'hall',
 'incred',
 'mcdougal',
 'ye',
 'hepburn',
 'einsteinian',
 'earmark',
 'sapl',
 'boar',
 'duan',
 'plain',
 'palfrey',
 'inflex',
 'like',
 'huzzah',
 'pepperoni',
 'bedtim',
 'nameabl',
 'attir',
 'tri',
 'edt',
 'chronographi',
 'optima',
 'ye',
 'pirogu',
 'diffus',
 'albeit']

In [11]:
from collections import Counter
import numpy as np

def build_dict(data, vocab_size = 4586):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    # TODO: Determine how often each word appears in `data`. Note that `data` is a list of sentences and that a
    #       sentence is a list of words.
    total_count= Counter()
    
    for i in range(len(data)):
        for word in data[i]:
            total_count[word] +=1
        
                
    word_count =total_count.most_common() # A dict storing the words that appear in the reviews along with how often they occur
    
        
    # TODO: Sort the words found in `data` so that sorted_words[0] is the most frequently appearing word and
    #       sorted_words[-1] is the least frequently appearing word.
    
    sorted_words = [item[0] for item in word_count]
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [12]:
word_dict_train= build_dict(words_train)
word_dict_test= build_dict(words_test)

list(word_dict_train)[:10]
print(len(word_dict_train))

4584


In [13]:
#Tokenization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Hyperparamaters
vocab_size= 1000
embedding_dim =16
max_length =100
trunc_type= 'post'
padding_type= 'post'
oov_tok ="<OOV>"


In [14]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(word_dict_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(word_dict_train)
training_padded= pad_sequences(sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)

testing_sequences = tokenizer.texts_to_sequences(word_dict_test)
testing_padded= pad_sequences(testing_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)




In [15]:
#Word Index
word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'enron': 2, 'ect': 3, 'subject': 4, 'vinc': 5, 'com': 6, 'hou': 7, 'pleas': 8, '2000': 9, 'kaminski': 10, 'would': 11, 'cc': 12, 'j': 13, 'thank': 14, '1': 15, 'time': 16, 'pm': 17, 'forward': 18, '2001': 19, '2': 20, 'inform': 21, 'work': 22, 'market': 23, 'price': 24, 'research': 25, 'get': 26, '10': 27, 'know': 28, 'may': 29, 'compani': 30, '3': 31, 'meet': 32, 'manag': 33, 'mail': 34, 'group': 35, 'busi': 36, 'use': 37, 'need': 38, 'e': 39, 'like': 40, 'new': 41, 'risk': 42, 'one': 43, 'email': 44, 'model': 45, 'messag': 46, 'energi': 47, 'power': 48, 'look': 49, 'regard': 50, 'us': 51, 'make': 52, '5': 53, '00': 54, 'let': 55, 'day': 56, 'year': 57, 'also': 58, 'interest': 59, 'http': 60, 'receiv': 61, 'develop': 62, '30': 63, 'call': 64, '4': 65, 'week': 66, '01': 67, 'shirley': 68, '11': 69, 'see': 70, 'contact': 71, 'houston': 72, '12': 73, 'want': 74, 'follow': 75, 'present': 76, 'edu': 77, 'list': 78, 'corp': 79, 'provid': 80, 'project': 81, 'program': 82, '04': 

In [16]:
#Building a model

model= tf.keras.Sequential([
                            tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                            tf.keras.layers.Dropout(0.5),
                            tf.keras.layers.Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'),
                            tf.keras.layers.Dropout(0.4),
                            tf.keras.layers.Conv1D(filters=16, kernel_size=2, padding='same', activation='relu'),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(6,activation='relu'),
                            tf.keras.layers.Dense(1, activation='sigmoid')

])

model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           16000     
_________________________________________________________________
dropout (Dropout)            (None, 100, 16)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 100, 32)           1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 16)           1040      
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 9